In [1]:
'''
Convert results from sql queries to csv files
'''
import sys

import os 
from os import environ as env

import mysql.connector
from mysql.connector import Error

import pandas as pd


In [2]:
if 'DB_PWD' in os.environ:
    del os.environ['DB_PWD']

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

## Warning!

We should use SQALchemy connection with pandas.

/home/jurra/work/projects/2022-Dyned/repos/dyned/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy


In [5]:
from sqlalchemy import create_engine

In [6]:
uri = f'mysql+mysqlconnector://{env["DB_USER"]}:{env["DB_PWD"]}@{env["DB_HOST"]}/{env["DB_NAME"]}'
db = create_engine(uri, echo=True)

In [7]:
def query_to_df(query):
    with db.connect() as connection:
        result = pd.read_sql_query(query, connection)
        return pd.DataFrame(result)

In [8]:
studies = query_to_df('''SELECT * FROM studies;''')
studies.head(1)

2022-04-13 17:42:22,235 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-04-13 17:42:22,236 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-13 17:42:22,242 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-04-13 17:42:22,244 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-13 17:42:22,246 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-04-13 17:42:22,247 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-13 17:42:22,257 INFO sqlalchemy.engine.Engine SELECT * FROM studies;
2022-04-13 17:42:22,259 INFO sqlalchemy.engine.Engine [raw sql] {}


,id,sort_position,code,public,publication_date,measurement_date,sources,people,publications,url,created_at,updated_at,has_shape_data
0,1,7,gdvv1984,0,1984,,,- J.F.M. Molenbroek\n- J.J. Houtkamp\n- A.K.C....,- Anthropometry of elderly people in the Nethe...,,2020-04-15 14:19:12,2020-04-15 14:25:12,0


In [9]:
tables = query_to_df("show tables;")
tables

2022-04-13 17:42:26,453 INFO sqlalchemy.engine.Engine show tables;
2022-04-13 17:42:26,454 INFO sqlalchemy.engine.Engine [raw sql] {}


,Tables_in_homestead
0,age_group_translations
1,age_groups
2,documents
3,download_events
4,individuals
5,locales
6,measure_group_translations
7,measure_groups
8,measure_translations
9,measurement_statistics


## Now we want to create an empty data frame with all the measures as columns

In [10]:
import json

In [11]:
with open ("./measures.json") as file:
    measures = json.loads(file.read())

In [12]:
df_measures = pd.DataFrame()
for measures_group in measures:
        for measure in measures_group['labels']:
                print("This is the id:", measure['id'], "This is the measure name:" ,measure['name_en'])
                break

This is the id: 1 This is the measure name: Reach height, standing
This is the id: 57 This is the measure name: Maximum gripping force
This is the id: 67 This is the measure name: Pronation
This is the id: 56 This is the measure name: Body mass
This is the id: 22 This is the measure name: Shoulder breadth (bi-deltoid)
This is the id: 55 This is the measure name: Ear length
This is the id: 43 This is the measure name: Hand length
This is the id: 41 This is the measure name: Foot length
This is the id: 12 This is the measure name: Shoulder-elbow length


In [26]:
all_individuals = pd.DataFrame()
for measures_group in measures:
        for measure in measures_group['labels']:
                measure = pd.DataFrame([], columns={ measure['id']})
                all_individuals = pd.concat([all_individuals, measure],ignore_index=True, axis=0)

all_individuals.insert(0, 'individual_id', pd.Series)
all_individuals.loc[len(all_individuals)] = 0
all_individuals

,individual_id,1,2,3,4,5,6,81,8,19,...,30,32,33,9,14,88,13,34,18,87
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Here we get all the measurements from the table

In [43]:
m = query_to_df("SELECT * FROM measurements")
m

2022-04-13 23:15:50,544 INFO sqlalchemy.engine.Engine SELECT * FROM measurements
2022-04-13 23:15:50,551 INFO sqlalchemy.engine.Engine [raw sql] {}


,id,measure_id,individual_id,study_id,value
0,1,56,1,1,103.00
1,2,29,1,1,525.00
2,3,43,1,1,198.00
3,4,47,1,1,89.00
4,5,22,1,1,484.00
...,...,...,...,...,...
243210,243211,98,9750,16,30.71
243211,243212,99,9750,16,22.39
243212,243213,100,9750,16,56.39
243213,243214,101,9750,16,63.03


In [48]:
import numpy as np

measure_id_unique=np.unique(m.loc[:,"measure_id"])
individual_id_unique=np.unique(m.loc[:,"individual_id"])

In [125]:
## Create a copy of the main dataframe to transform
m2data=m.copy()
## Make the new index the individuals ID for easier handling 
m2data.index=m.loc[:,"individual_id"]

Int64Index([   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            ...
            9750, 9750, 9750, 9750, 9750, 9750, 9750, 9750, 9750, 9750],
           dtype='int64', name='individual_id', length=243215)

In [74]:
## Dataframe where the columns are all the ordered ID measurements and the rows are all the ordered individuals ID 
standard_dataframe=pd.DataFrame(columns=np.arange(0,len(measure_id_unique)),
index=np.arange(0,len(individual_id_unique)))

standard_dataframe.columns=measure_id_unique

In [135]:
# Running over the columns of the newly created dataframe 
for col in measure_id_unique:
    # extract all the individuals ID per measure ID
    tmp_index=m2data[m2data.loc[:,"measure_id"]==col].index
    # extract all the values per measure ID
    tmp_value=m2data[m2data.loc[:,"measure_id"]==col]["value"].tolist()

    ## Assign each value to each individual ID in the dataframe 
    for i in np.arange(0,len(tmp_index)):
        tmp_index_unique=np.where(individual_id_unique==tmp_index[i])
        standard_dataframe.loc[tmp_index_unique[0],col]=tmp_value[i]

In [136]:
standard_dataframe

,1,2,3,4,5,6,7,9,10,11,...,96,97,98,99,100,101,102,103,104,105
0,1839.0,1745.0,1600.0,650.0,1150.0,836.0,221.0,170.0,67.0,238.0,...,17.426,35.319,38.37,32.18,60.86,76.29,108.534,147.436,15.378,36.657
1,1787.0,1598.0,1510.0,536.0,737.0,778.0,209.0,109.0,71.0,228.0,...,20.509,35.725,40.99,25.44,57.54,69.26,112.778,156.471,14.9,37.749
2,1678.0,1598.0,1510.0,557.0,1115.0,748.0,225.0,110.0,80.0,238.0,...,20.322,33.812,34.19,22.84,60.92,73.8,118.826,138.017,14.563,41.143
3,1787.0,1638.0,1519.0,558.0,717.0,714.0,214.0,102.0,74.0,236.0,...,18.431,33.392,28.68,19.88,52.1,62.3,116.531,140.21,12.022,39.222
4,1678.0,1501.0,1405.0,497.0,1144.0,695.0,205.0,110.0,69.0,226.0,...,21.093,40.972,37.77,19.39,60.78,70.62,135.585,161.778,15.463,40.678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9740,NaN,1287.0,NaN,NaN,NaN,NaN,193.0,NaN,NaN,NaN,...,16.667,30.62,43.04,26.61,71.68,84.03,NaN,NaN,NaN,NaN
9741,NaN,1281.0,NaN,NaN,NaN,NaN,198.0,NaN,NaN,NaN,...,14.6,27.19,43.73,26.21,75.8,84.7,NaN,NaN,NaN,NaN
9742,NaN,1254.0,NaN,NaN,NaN,NaN,188.0,NaN,NaN,NaN,...,19.016,31.11,37.47,26.06,72.09,79.14,NaN,NaN,NaN,NaN
9743,NaN,1203.0,NaN,NaN,NaN,NaN,199.0,NaN,NaN,NaN,...,16.372,28.84,39.12,25.79,76.46,85.25,NaN,NaN,NaN,NaN
